#### Starting out with Tensorflow with TensorBoard
TensorBoard is a visualization Board to understand what is going on in your Tensorflow programs. **Learning to use TensorBoard early and often will make using Tensorflow that much more enjoyable and productive**

In [10]:
import tensorflow as tf

In [7]:
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
with tf.Session() as sess:
    #Visualizing with tensorboard
    writer = tf.summary.FileWriter('./logs_dir', sess.graph)
    print(sess.run(x))

#Close the writer when you are done
writer.close()

#Other note: After creater the writer, the session graph is placed inside a logs_dir as stipulated and then you can run
#    'tensorboard --logdir="<views directory>" e.g "/.logs_dir"
# It's a good thing to always name your variable for good visualization
#E.g a = tf.constant(2, name="a")

5


In [5]:
#Let's do more: Assuming we have a function f = x2y + y + 2 and we are to use tensorflow to compute f given x = 3 and y = 4
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

#Initialize global variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    writer = tf.summary.FileWriter('./logs_dir', sess.graph)
    result = f.eval()

writer.close()

In [6]:
result

42

In [30]:
#Running Linear Regression with Tensorflow
import numpy as np
import os
import pandas as pd


FILENAME = 'housing.csv'
HOUSING_PATH = "datasets/housing"
def load_housing_data(housing_path=HOUSING_PATH, csv_filename=FILENAME):
    """
    Function to load csv file given a housing, path
    """
    csv_path = os.path.join(housing_path, csv_filename)
    return pd.read_csv(csv_path)


housing = load_housing_data()

In [28]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [25]:
housing.isnull().any()

longitude             False
latitude              False
housing_median_age    False
total_rooms           False
total_bedrooms         True
population            False
households            False
median_income         False
dtype: bool

In [31]:
median = housing.total_bedrooms.median()
housing["total_bedrooms"].fillna(median, inplace=True)

In [32]:
housing_target = housing['median_house_value']
housing_target = pd.Series.as_matrix(housing_target)
housing.drop(['ocean_proximity', 'median_house_value'],axis=1, inplace=True)
housing_matrix = pd.DataFrame.as_matrix(housing)

m, n =housing.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing_matrix]

In [48]:
#USING METHOD TWO TO FETCH THE DATA
import tarfile

def fetch_california_housing(tgz_filepath):
    with open(tgz_filepath, "rb") as f:
        fileobj = tarfile.open(mode="r:gz", fileobj=f).extractfile('CaliforniaHousing/cal_housing.data')
        cal_housing = np.loadtxt(fileobj, delimiter=',')
    columns_index = [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]
    y, X = cal_housing[:, 0], cal_housing[:, 1:]
    # avg rooms = total rooms / households
    X[:, 2] /= X[:, 5]
    # avg bed rooms = total bed rooms / households
    X[:, 3] /= X[:, 5]
    # avg occupancy = population / households
    X[:, 5] = X[:, 4] / X[:, 5]
    # Target in units of 100,000
    y = y / 100000.0
    return X, y

X_1, y_1 = fetch_california_housing("datasets/housing/cal_housing.tgz")

In [37]:
tf.reset_default_graph()

X = tf.constant(housing_data_plus_bias, dtype=tf.float64, name="X")
y = tf.constant(housing_target.reshape(-1, 1), dtype=tf.float64, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    result = theta.eval()

print(result)

[[ -3.57011806e+06]
 [ -4.26104026e+04]
 [ -4.24754782e+04]
 [  1.14445085e+03]
 [ -6.62091740e+00]
 [  8.11609666e+01]
 [ -3.98732002e+01]
 [  7.93047225e+01]
 [  3.97522237e+04]]


In [51]:
X_1.shape, housing_data_plus_bias.shape

((20640, 8), (20640, 9))

In [54]:
y_1.shape, housing_target.shape

((20640,), (20640,))

In [52]:
#Result from California house from site
#Prepare X by fixing in bias
m, n = X_1.shape
X_with_bias = np.c_[np.ones((m, 1)), X_1]

tf.reset_default_graph()
X = tf.constant(X_with_bias, dtype=tf.float64, name="X")
y = tf.constant(y_1.reshape(-1, 1), dtype=tf.float64, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    result = theta.eval()

print(result)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]
